# Submitting Cirq Circuits to Azure Quantum with the QDK

This notebook shows how to take a Cirq `Circuit`, export it to OpenQASM 3, compile that OpenQASM 3 source to QIR using the Microsoft Quantum Development Kit (QDK) Python APIs, and submit it as a job to an Azure Quantum target. The focus is on the core submission flow (no parameterization complexity).

The workflow demonstrated here:

1. Build (or load) a Cirq `Circuit`.
2. Convert it to OpenQASM 3 text via `circuit.to_qasm(version="3.0")`.
3. Compile the OpenQASM 3 source to QIR with `qdk.openqasm.compile`.
4. Connect to (or create) an Azure Quantum workspace using `qdk.azure.Workspace`.
5. Pick a target (e.g., a simulator like `rigetti.sim.qvm`).
6. Submit the QIR payload and retrieve measurement results.

## Prerequisites
Ensure the `qdk` package (with Azure + Cirq extras) and `cirq` are installed. If not, install dependencies below.

In [ ]:
%pip install qdk[azure,cirq]

After installing, restart the kernel if necessary. Verify imports:

In [ ]:
import cirq, qdk, qdk.azure  # should import without errors

## Submitting a simple Cirq circuit
We'll build a small circuit creating a superposition on one qubit and flipping another, then measuring both. Afterwards we submit it to an Azure Quantum target.

In [ ]:
# Build a simple circuit
q0, q1 = cirq.LineQubit.range(2)
simple_circuit = cirq.Circuit(
    cirq.H(q0),
    cirq.measure(q0, key='m0'),
    cirq.X(q1),
    cirq.measure(q1, key='m1'),
)
print(simple_circuit)

## Configure Azure Quantum workspace connection
Replace the placeholder values below with your own subscription, resource group, workspace name, and location.

In [ ]:
subscription_id = 'xxxxxxxx-xxxx-xxxx-xxxx-xxxxxxxxxxxx'
resource_group = 'myresourcegroup'
workspace_name = 'myworkspace'
location = 'westus'

In [ ]:
from qdk.openqasm import compile
from qdk.azure import Workspace
from qdk import TargetProfile

def submit_cirq_circuit_to_azure(circuit: cirq.Circuit, target_name: str, name: str, shots: int = 100):
    # 1. Export to OpenQASM 3
    qasm3_str = circuit.to_qasm(version='3.0')
    # 2. Compile to QIR with a base target profile
    qir = compile(qasm3_str, target_profile=TargetProfile.Base)
    # 3. Connect workspace
    workspace = Workspace(
        subscription_id=subscription_id,
        resource_group=resource_group,
        name=workspace_name,
        location=location,
    )
    # 4. Select target (string e.g. 'rigetti.sim.qvm' or other available target)
    target = workspace.get_targets(target_name)
    # 5. Submit QIR payload
    job = target.submit(qir, name, shots=shots)
    return job.get_results()

### Submit the simple circuit
(Make sure you changed the workspace credentials above.)

In [ ]:
# Uncomment after setting workspace credentials
# results = submit_cirq_circuit_to_azure(simple_circuit, 'rigetti.sim.qvm', 'cirq-simple-job')
# print(results)

## Notes
- Ensure all measurement keys appear before any classical condition usage if you introduce classical controls.
- Multi-target or custom gates may need full decomposition before submission if they produce unsupported classical constructs.
- Keep circuits minimal when first validating workspace access; expand complexity gradually.
- Complex classical control expressions created with SymPy symbols are not supported when generating QASM, and therefore can't be submitted to Azure using the method shown in this sample.

## Exercises
1. Add another qubit prepared with `cirq.Y` then measure it; resubmit.
2. Swap the target to a different available simulator or hardware backend.
3. Print the OpenQASM text and inspect how measurements are represented.
4. Add a simple classical-controlled gate (e.g. apply `cirq.Z(q0)` conditioned on `m0`) and verify QASM export still succeeds.